# Статистики за корона вирусот во Македонија

In [2]:
import pandas as pd
# import nbinteract as nbi
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, fixed

%matplotlib inline

In [77]:
all_data = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
all_data = all_data.replace('North Macedonia', 'Macedonia')
all_data.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,53,59,70,75,82,114,147,177,212,272
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,581,639,639,701,773,839,825,878,889,924
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,160,178,178,200,212,226,243,266,313,345
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,129,149,149,197,238,428,566,673,790,900


In [96]:
def get_data_by_country_name(data, country_column, country_name):
    return data.loc[data[country_column] == country_name]


def convert_data_in_day_count_larger_than_zero_format(data, id_vars, value_vars, var_name, value_name):
    converted = pd.melt(data, id_vars=id_vars, value_vars=value_vars, var_name=var_name, value_name=value_name)
    converted = converted.loc[converted[value_name] > 0].reset_index(drop=True)
    converted.insert(0, 'artificial_day', range(1, 1 + len(converted)))
    return converted.loc[converted[value_name] > 0].reset_index(drop=True)

def get_data_for_country(data, name):
    res = get_data_by_country_name(data=data, country_column='Country/Region', country_name=name)
    res = convert_data_in_day_count_larger_than_zero_format(data=res, id_vars=res.columns[1 : 4], value_vars=res.columns[4:], var_name='date', value_name='count')
    return res


In [97]:
def get_data_for_countries(data, countries):
    data = [get_data_for_country(data, country) for country in countries]
#     interesting_data = pd.concat([country_data.iloc[:comparable_days_count] for country_data in data])
    return data

def draw_line_plot(day_range, countries, data):
    figure = plt.figure(num='figure', figsize=(15,10))
    from_day = day_range[0]
    to_day = day_range[1]
    data = get_data_for_countries(data, countries)
    interesting_data = pd.concat([country_data.loc[(country_data['artificial_day'] >= from_day) & 
                                                   (country_data['artificial_day'] <= to_day)] 
                                  for country_data in data])

    ax = plt.axes()
    line_plot = sns.lineplot(x="artificial_day", y="count", hue='Country/Region',
                             data=interesting_data, ax = ax)

    line_plot.set(xlabel='Релативни денови од кога се појавил првиот случај во таа држава',
                  ylabel='Број на заразени')
    ax.legend().texts[0].set_text("Држава:")
    line_plot.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    line_plot.yaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    
    return line_plot


In [98]:
data = convert_data_in_day_count_larger_than_zero_format(data=all_data, id_vars=all_data.columns[1 : 4],
                                                         value_vars=all_data.columns[4:], var_name='date',
                                                         value_name='count')
day_range_chooser = widgets.IntRangeSlider(
    value=[1, 20],
    min=1,
    max=len(all_data.columns) - 4,
    step=1,
    description='Денови:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

country_selector = widgets.SelectMultiple(
    options=all_data['Country/Region'].unique(),
    value=['Macedonia', 'Serbia', 'Albania', 'Greece', 'Bulgaria'],
    description='Држави:',
    disabled=False
)


interact(draw_line_plot, day_range=day_range_chooser, countries=country_selector, data=fixed(all_data))

interactive(children=(IntRangeSlider(value=(1, 20), continuous_update=False, description='Денови:', max=58, mi…

<function __main__.draw_line_plot(day_range, countries, data)>